In [1]:
import numpy as np
import pandas as pd

# Análisis de la Data

In [ ]:
df = pd.read_csv()

Pasos:

1° duplicados, NaN, filtrar "cancelados", filtrar clientes que desde 2019 no compran

2° Colección: SS -- 01-05-2016   -   FW -- 01-11-2016
             Agregar 2 nueva columna SS o FW
   Referencia -- Familia -- "encoding" (agregar columnas)
   Total piezas (y - target)
   país productor -- corregir texto con distinta escritura (Servia, Camboya) 
                  -- encoding
   Cliente -- encoding
   Plataforma -- encoding